# Sample satellite data

Some graphical tests of current primary functions using the sample satellite data (MSG SEVIRI brightness temperature).

In [ ]:
import os
import warnings

import cartopy.io
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import regionmask
import xarray as xr
from xrframes import Frames

import tams

warnings.filterwarnings("ignore", category=cartopy.io.DownloadWarning)

%matplotlib inline

xr.set_options(display_expand_data=False)

## Load data

In [ ]:
tb = tams.data.open_example("msg-tb")["tb"].isel(time=slice(4)).load()
tb

In [ ]:
tb.isel(time=0).plot(x="lon", y="lat", size=5, aspect=2.5);

In [ ]:
pr = tams.data.open_example("imerg")["pr"].resample(time="2h").mean().load()
pr

In [ ]:
pr.isel(time=0).plot(size=4.5, aspect=2.5, robust=True);

## Identify cloud elements (CEs)

In [ ]:
times = tb.time
ces = tams.identify(tb)
ces[0].head()

In [ ]:
# Simple plot to test 219 matching
m, n = 0, 1  # time, contour #
fig, ax = plt.subplots()
c = ces[m].iloc[[n]]
c.plot(ax=ax)
c.core.plot(color="red", ax=ax, alpha=0.4);

## Track CE groups between times

In [ ]:
ce = tams.track(ces, times, u_projection=-5).reset_index(drop=True)
ce.head()

In [ ]:
tams.plot_tracked(ce)

In [ ]:
size = 2.5
cx = cy = 3
vmin_tb, vmax_tb = 190, 300
vmin_pr, vmax_pr = 0.1, 20

if cx > 1 or cy > 1:
    tb_ = tb.coarsen(x=cx, y=cy, boundary="trim").mean()
else:
    tb_ = tb

extent = [-40, 50, 0, 20]

x0, x1, y0, y1 = extent
aspect = (x1 - x0) / (y1 - y0)
proj = ccrs.Mercator()
tran = ccrs.PlateCarree()

def plot(tb_i):
    fig = plt.figure(figsize=(size * aspect, size + 1))
    gs = fig.add_gridspec(
        2, 2,
        width_ratios=(1, 1), height_ratios=(aspect * 2 + 1, 1),
        left=0.1, right=0.9, bottom=0.1, top=0.9,
        wspace=0.05, hspace=0.18,
    )

    ax = fig.add_subplot(gs[0, :], projection=proj)
    ax.set_extent(extent, crs=tran)
    ax.gridlines(draw_labels=True)
    ax.coastlines(color="orange", alpha=0.5)

    ax2 = fig.add_subplot(gs[1, 0])
    ax3 = fig.add_subplot(gs[1, 1])

    t = pd.Timestamp(tb_i.time.item())

    # Background -- CTT
    tb_i.plot(
        x="lon", y="lat",
        cmap="gray_r", ax=ax, cbar_ax=ax2,
        transform=tran,
        cbar_kwargs=dict(orientation="horizontal"),
        vmin=vmin_tb, vmax=vmax_tb, extend="both",
    )

    # CEs with colored precip
    pr_i = pr.sel(time=t)
    shapes = ce.query("time == @t")[["geometry"]]
    shapes.plot(ax=ax, fc="none", ec="purple", transform=tran)
    regions = regionmask.from_geopandas(shapes, overlap=False)
    # NOTE: regionmask reports some overlap does exist
    mask = regions.mask(pr_i)
    masked = pr_i.where((mask >= 0) & (pr_i >= vmin_pr))
    masked.plot.pcolormesh(
        x="lon", y="lat",
        ax=ax, cbar_ax=ax3, transform=tran, alpha=0.6,
        cbar_kwargs=dict(orientation="horizontal"),
        vmin=vmin_pr, vmax=vmax_pr, extend="both",
    )

    # Tracks up to this time
    for _, g in ce.groupby("mcs_id"):
        g_ = g[g.time <= t].dissolve("itime")
        c = g_.to_crs("EPSG:32663").centroid.to_crs("EPSG:4326")
        ax.plot(c.x, c.y, ".-", c="r", lw=2, alpha=0.4, transform=tran)
        c_t = c[g_.time == t]
        if not c_t.empty:
            ax.plot(c_t.x, c_t.y, ".", c="r", ms=8, transform=tran)

    ax.set_title("")
    ax.set_title(f"{t:%Y-%m-%d %HZ}", loc="left", size=11)

frames = Frames(tb_, plot, dim="time")
frames.write(dpi=120)
frames.to_gif("./tb.gif", fps=1, magick="READTHEDOCS" not in os.environ)

In [ ]:
frames.display()

## Classify

In [ ]:
ce = tams.classify(ce)
ce.head()